In [ ]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

In [82]:
# initializing pandas dataframe
cfa_readings_df = pd.DataFrame(columns=['topic_name', 'year', 'level', 'introduction', 'learning_outcome', 'summary_page_link', 'pdf_file_Link'])

In [83]:
base_url = "https://www.cfainstitute.org/en/membership/professional-development/refresher-readings"

id_ = "".join(re.findall(r"\d+", base_url))
# pages = 1
pages = 23
page_size = 10

# Set up the Selenium webdriver
driver = webdriver.Chrome()
driver1 = webdriver.Chrome()

# for i in range(1, pages + 1):
for i in range(0, pages):

    print(f"Scraping page {i}")
    url = f"{base_url}#first={i}0&sort=%40refreadingcurriculumyear%20descending"
    driver.get(url)
    # Wait for the dynamic content to load
    time.sleep(5)
    # Once the content is loaded, get the page source
    page_source = driver.page_source
    # Collect HTML data from this page
    parsed_content = BeautifulSoup(page_source, 'html.parser')

    for div in parsed_content.find_all('div', attrs={"class":"coveo-list-layout CoveoResult"}):
        url_extension = div.find('a', class_= 'CoveoResultLink')["href"]
        driver1.get(url_extension)
        time.sleep(5)
        # # Once the content is loaded, get the page source
        extended_page_source = driver1.page_source
        ext_parsed_content = BeautifulSoup(extended_page_source, 'html.parser')
        
        topic_name = div.find('a',class_='CoveoResultLink').get_text()
        
        try:
            year= ext_parsed_content.find('span', class_ = "content-utility-curriculum").get_text().strip()[:4]
        except Exception as e:
            print("Exception: ",e)
            year = None
        
        level = ext_parsed_content.find('span', class_="content-utility-topic").get_text().strip().strip("Level")
        
        try:
            pdf_section = ext_parsed_content.find('section', class_="primary-asset login-required")
            if pdf_section:
                for child_tag in pdf_section.find_all('a'):
                    child_tag_text = child_tag.get_text()
                    if child_tag_text=="Download the full reading (PDF)":
                        pdf_link = "https://www.cfainstitute.org"+ child_tag['href']
                        
                introduction_section = pdf_section.findNext('section')
            else:
                raise Exception("No PDF section available")
            
        except Exception as e:
            print("Exception: ",e)
            pdf_link=None
            introduction_section = ext_parsed_content.find("div", class_="content-utility").findNext('section')
        
        intro_list = []
        for child_tag in introduction_section:
            intro_list.append(child_tag.get_text(strip=True))
        introduction = " ".join(intro_list)
        
        learning_outcome_section = introduction_section.findNext('section')
        learning_outcome_list = []
        for child_tag in learning_outcome_section:
            learning_outcome_list.append(child_tag.get_text(strip=True))
        learning_outcome = " ".join(learning_outcome_list)
        
        cfa_readings_df.loc[len(cfa_readings_df)] = {'topic_name':topic_name, 'year': year, 'level': level,'introduction':introduction.replace("Introduction",""),
                                                     'learning_outcome':learning_outcome, 'summary_page_link':url_extension,
                                                     'pdf_file_Link': pdf_link}

# Close the Selenium webdriver
driver.quit()
driver1.quit()
cfa_readings_df.to_csv('scraped_data.csv')
cfa_readings_df.head()
    

Scraping page 0
Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
Exception:  No PDF section available
Scraping page 11
Exception:  No PDF section available
Scraping page 12
Scraping page 13
Scraping page 14
Exception:  No PDF section available
Exception:  'NoneType' object has no attribute 'get_text'
Exception:  No PDF section available
Exception:  'NoneType' object has no attribute 'get_text'
Exception:  No PDF section available
Exception:  'NoneType' object has no attribute 'get_text'
Exception:  No PDF section available
Exception:  'NoneType' object has no attribute 'get_text'
Exception:  No PDF section available
Exception:  No PDF section available
Exception:  No PDF section available
Exception:  No PDF section available
Scraping page 15
Exception:  No PDF section available
Exception:  No PDF section available
Exception:  No PDF section available
Exception:  No PDF sectio

,topic_name,year,level,introduction,learning_outcome,summary_page_link,pdf_file_Link
0,Time-Series Analysis,2024,Level II,"As financial analysts, we often use time-s...",The member should be able to: calculate and...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
1,Credit Analysis Models,2024,Level II,Credit analysis plays an important role in...,The member should be able to: explain expec...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
2,Introduction to Alternative Investments,2023,Level I,"In this section, we explain what alternati...",The member should be able to: describe type...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
3,Credit Default Swaps,2024,Level II,Derivative instruments in which the underl...,The member should be able to: describe cred...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
4,Valuation of Contingent Claims,2024,Level II,A contingent claim is a derivative instrum...,The member should be able to: describe and ...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...


In [84]:
cfa_readings_df

,topic_name,year,level,introduction,learning_outcome,summary_page_link,pdf_file_Link
0,Time-Series Analysis,2024,Level II,"As financial analysts, we often use time-s...",The member should be able to: calculate and...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
1,Credit Analysis Models,2024,Level II,Credit analysis plays an important role in...,The member should be able to: explain expec...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
2,Introduction to Alternative Investments,2023,Level I,"In this section, we explain what alternati...",The member should be able to: describe type...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
3,Credit Default Swaps,2024,Level II,Derivative instruments in which the underl...,The member should be able to: describe cred...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
4,Valuation of Contingent Claims,2024,Level II,A contingent claim is a derivative instrum...,The member should be able to: describe and ...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
...,...,...,...,...,...,...,...
219,Fixed-Income Cash Flows and Types,2024,Level I,Overview A fixed-income instrument’s cash fl...,1\nPLRecord PL creditManage your Profession...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
220,"Private Capital, Real Estate, Infrastructure, ...",2023,Level I,Private capital is the broad term for fund...,The member should be able to: explain inves...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
221,Extensions of Multiple Regression,2024,Level II,,The member should be able to: describe infl...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
222,Pricing and Valuation of Forward Contracts and...,2024,Level I,Earlier lessons introduced forward commitm...,The member should be able to: explain how t...,https://www.cfainstitute.org/membership/profes...,https://www.cfainstitute.org/-/media/documents...
